In [3]:
import numpy as np
import math
from fractions import Fraction
from tqdm import tqdm
import multiprocessing as mp
from haversine import haversine, Unit
import matplotlib.pyplot as plt
import os
import random
import matplotlib
from scipy.sparse import lil_matrix, csr_matrix
import gc
import time
from multiprocessing import shared_memory

In [20]:
# Astronomy-related constants

TE = 24 * 3600  # One Earth sidereal day in seconds

RE = 6371e3  # Mean radius of Earth in meters

u = 3.986e14  # Standard gravitational parameter (μ = GM) in m^3/s^2

K = RE / pow(u, 1/3) * pow(2 * np.pi, 2/3)  

eps = 25 * np.pi / 180  # Maximum elevation angle for ground access, in radians (25°)

In [21]:
# Get all files (excluding Jupyter checkpoints)
def get_allfile(path):
    return [os.path.join(path, f) for f in os.listdir(path) if "ipynb_checkpoints" not in f]

# Compute satellite orbital period (h in meters)
def satellite_period(h):
    a = RE + h
    T = float(2 * np.pi * pow(a**3 / u, 0.5))
    return T

# Compute coverage angle eta (in radians)
def coverage_eta(T):
    eta = math.acos(K * math.cos(eps) / pow(T, 2/3)) - eps
    return eta

# Approximate an integer ratio of a and b
def approximate_ratio(a, b, precision=1e-3):
    if b == 0:
        raise ValueError("Denominator cannot be zero")
    ratio = Fraction(a, b).limit_denominator(int(1 / precision))
    return ratio.numerator, ratio.denominator

# Convert orbital parameters to longitude/latitude
def alpha_gamma_to_lambda_phi(alpha, gamma, inc):
    phi = math.asin(math.sin(inc) * math.sin(gamma))
    temp = math.atan2(math.cos(inc) * math.sin(gamma), math.cos(gamma))
    lamb = (temp + alpha) % (2 * math.pi)
    if lamb > math.pi:
        lamb -= 2 * math.pi
    return lamb, phi

# Check whether satellite covers a ground cell
def is_cover(l_sat, p_sat, l_cell, p_cell, eta):
    # If latitude/longitude range overflows, fallback to haversine distance
    if (p_sat - 2*eta) < -np.pi/2 or (p_sat + 2*eta) > np.pi/2 or (l_sat - 2*eta) < -np.pi or (l_sat + 2*eta) > np.pi:
        d = haversine(
            (np.degrees(p_sat), np.degrees(l_sat)),
            (np.degrees(p_cell), np.degrees(l_cell)),
            unit=Unit.METERS
        )
        return d <= eta * RE
    else:
        # Fast bounding-box filter + haversine check
        if (p_sat - 2*eta) <= p_cell <= (p_sat + 2*eta) and (l_sat - 2*eta) <= l_cell <= (l_sat + 2*eta):
            d = haversine(
                (np.degrees(p_sat), np.degrees(l_sat)),
                (np.degrees(p_cell), np.degrees(l_cell)),
                unit=Unit.METERS
            )
            return d <= eta * RE
    return False


In [22]:
def cal_supply_all_cell(h, beta, a0):
    """
    Compute all ground coverage slots for a single stable satellite track.

    Args:
        h: Satellite altitude (km)
        beta: Inclination (rad)
        a0: Ascending node longitude (rad)

    Returns:
        total_supply: list of [covered_cells, lon, lat] for each slot
    """
    T = satellite_period(h * 1e3)
    p, q = approximate_ratio(int(T), TE, precision=1e-3)
    eta = coverage_eta(T)
    step = 2 * eta
    inc = beta

    total_supply = []
    slot_num = int(2 * np.pi * q / step)

    for i in range(slot_num):
        t = step * i / (2 * np.pi / T)
        a_sat = (a0 - (2 * np.pi / TE) * t) % (2 * np.pi)
        b_sat = (step * i) % (2 * np.pi)
        l_sat, p_sat = alpha_gamma_to_lambda_phi(a_sat, b_sat, inc)

        r_set = []
        for idx, demand in enumerate(demand_list):
            l_cell = demand['lat_lon'][1]
            p_cell = demand['lat_lon'][0]
            if is_cover(l_sat, p_sat, l_cell, p_cell, eta):
                S = S_set[idx]
                r_set.append([idx, S, demand['density']])

        if r_set:
            S_sum = np.sum([item[1] for item in r_set])
            factor = user_per_sat / S_sum
            r_set = [[item[0], item[1] * factor] for item in r_set]
            total_supply.append([r_set, l_sat, p_sat])

    return np.asarray(total_supply, dtype="object")


In [23]:
# Recover full-time user coverage from time-shifted satellites
def recover_fulltime(data):
    random_numbers, cover, n_length = data
    selected_cover = lil_matrix((1, n_length), dtype=np.float64)
    sat_location = []

    for t in range(time_split):
        for num in random_numbers:
            slot = (num + t) % time_split
            sat_location.append([cover[slot][1], cover[slot][2]])
            for idx, users in cover[slot][0]:
                selected_cover[0, t * len(demand_list) + idx] += users

    return [selected_cover.tocsr(), sat_location]

# Generate one base configuration with full-time coverage
def generate_baseset(args):
    file, random_numbers, sat_num = args
    file_param = file.split("/")[1].split("_")
    h = float(file_param[0])
    beta = float(file_param[1])
    alpha0 = float(file_param[2].split(".npy")[0])
    param = [h, beta, alpha0]
    cover = cal_supply_all_cell(h,beta,alpha0)
    fulltime_cover = recover_fulltime([random_numbers, cover, n_length])
    
    return [param, random_numbers, fulltime_cover, sat_num]


In [24]:
def compute_dot(args, show=False):
    """
    Compute dot product between a candidate's full-time coverage vector and residual demand R.
    
    Args:
        args: Tuple of (R, cid), where
            - R: residual demand vector
            - cid: candidate index
        show: optional debug flag

    Returns:
        List: [dot value, cid]
    """
    R, cid = args
    fulltime_cover = all_cover[cid][2][0]  # csr sparse matrix
    dot = fulltime_cover @ R  # single value (array of shape (1,))
    return [dot[0], cid]

def update_residual(R, cid):
    """
    Subtract selected candidate's contribution from residual demand R.

    Args:
        R: current residual vector
        cid: candidate ID

    Returns:
        Updated residual R (clipped at 0)
    """
    cover_csr = all_cover[cid][2][0]
    sat_num = all_cover[cid][3]
    R[cover_csr.indices] -= (cover_csr.data * sat_num)
    R = np.maximum(R, 0)  # ensure no negative values
    return R

def find_max_in_chunk(chunk):
    """
    Find the max value and corresponding candidate index in a chunk of dot results.

    Args:
        chunk: array-like of shape (N, 2) -> [dot_value, cid]

    Returns:
        [max_dot_value, cid]
    """
    chunk = np.array(chunk)
    max_value = np.max(chunk[:, 0])
    local_index = np.argmax(chunk[:, 0])
    max_index = chunk[local_index][1]
    return [max_value, int(max_index)]

# Starlink customer

In [32]:
sat_height = 573  # Satellite altitude in km
base_cover_path = f"candidate_orbits_starlink_customer_{sat_height}/"
time_split = 308  # Number of time slots

# Load user demand and convert lat/lon to radians
demand_list = np.load("../../Dataset/Demand/starlink_supply_2025_01_01_99449.npy", allow_pickle=True)
for demand in demand_list:
    demand['lat_lon'] = [np.radians(demand['lat_lon'][0]), np.radians(demand['lat_lon'][1])]

cell_size = 4  # Grid size in degrees
user_per_sat = 960

# Compute area of each demand cell
S_set = []
for demand in demand_list:
    p_cell = demand['lat_lon'][0]  # latitude in radians
    h = RE * np.radians(cell_size)

    L1 = RE * np.sin(np.pi / 2 - abs(p_cell)) * np.radians(cell_size)
    tmp = abs(p_cell - np.radians(cell_size))
    L2 = RE * np.sin(np.pi / 2 - min(tmp, np.pi/2)) * np.radians(cell_size)

    # Special case for South Pole
    if tmp > np.pi / 2:
        h = RE * abs(p_cell + np.pi / 2)

    S = 0.5 * h * (L1 + L2)
    S_set.append(S)


In [33]:
selected_params=np.load("data/Tinyleo_for_starlink_demand.npy",allow_pickle=True)
print(len(selected_params))
files=[]
sat_nums=0
for s_tmp in selected_params:
    param_tmp,slots,satmaps,sat_num=s_tmp
    tmp=base_cover_path+str(int(param_tmp[0]))+"_"+str(param_tmp[1])+"_"+str(param_tmp[2])+".npy"
    files.append([tmp,slots,sat_num])
    sat_nums+=sat_num
print(files[0])
print(sat_nums)

1762
['candidate_orbits_starlink_customer_573/573_2.234021442552742_1.53588974175501.npy', [87], 1]
1762


In [34]:
# files=get_allfile(base_cover_path)
# print(files[0])
all_cover=[]
n_length=len(demand_list)*time_split

with mp.Pool(processes=100) as pool:
    all_cover = list(tqdm(pool.imap(generate_baseset, files), total=len(files), mininterval=10,maxinterval=20))
    pool.close()  # Prevents any more tasks from being submitted to the pool
    pool.join()  # Wait for the worker processes to exit

100%|██████████| 1762/1762 [00:42<00:00, 41.56it/s]


In [35]:
num_processes = 20

# Initialize residual demand vector R_2 over all time slots
tmp_R = [demand['density'] for demand in demand_list]
R_2 = np.array(tmp_R * time_split).reshape(-1).copy()
R_sum_2 = np.sum(R_2)
print("Initial residual sum:", R_sum_2)

# Free memory
del tmp_R

process = []
for idx in range(len(all_cover)):
    R_2 = update_residual(R_2, idx)  # Subtract current satellite's supply
    covered = R_sum_2 - np.sum(R_2)  # Demand covered in this step
    process.append([all_cover[idx][3], covered])
    R_sum_2 = np.sum(R_2)
    print("Residual sum after", idx + 1, ":", R_sum_2)

# Final residual diagnostics
print(f"Remaining nonzero entries = {np.count_nonzero(R_2)}, "
      f"Max = {np.max(R_2):.3f}, Mean = {np.mean(R_2):.6f}")


Initial residual sum: 30630530.279174425
Residual sum after 1 : 30574305.861058682
Residual sum after 2 : 30518081.442942902
Residual sum after 3 : 30461857.024827156
Residual sum after 4 : 30405632.606711384
Residual sum after 5 : 30349408.188595634
Residual sum after 6 : 30293183.770479877
Residual sum after 7 : 30236959.352364108
Residual sum after 8 : 30180734.93424833
Residual sum after 9 : 30124510.516132567
Residual sum after 10 : 30068286.098016772
Residual sum after 11 : 30012061.67990105
Residual sum after 12 : 29955837.26178527
Residual sum after 13 : 29899612.843669523
Residual sum after 14 : 29843388.425553747
Residual sum after 15 : 29787164.00743797
Residual sum after 16 : 29730939.589322235
Residual sum after 17 : 29674715.171206445
Residual sum after 18 : 29618490.75309069
Residual sum after 19 : 29562266.334974933
Residual sum after 20 : 29506122.399195466
Residual sum after 21 : 29449978.46341601
Residual sum after 22 : 29393834.527636528
Residual sum after 23 : 2933

In [36]:
print(f"Remaining nonzero entries = {np.count_nonzero(R_2)}, Max = {np.max(R_2):.2f}, Mean = {np.mean(R_2):.2f}")

Remaining nonzero entries = 38, Max = 2.84, Mean = 0.00


In [37]:
plt_data=[0]
r_sum=sum([demand['density'] for demand in demand_list])*time_split
for item in process:
    for _ in range(item[0]):
        v=item[1]/item[0]/r_sum*100+plt_data[-1]
        plt_data.append(v)
np.save("data/network_availability_official_demand.npy",plt_data)

In [60]:
time_shift=0
total_supply=[0]*len(demand_list)
for cover in tqdm(all_cover):
    cover_csr = cover[2][0]
    sat_num = cover[3]
    total_supply += cover_csr.toarray().flatten()[len(demand_list)*time_shift:len(demand_list)*(time_shift+1)] * sat_num  

100%|██████████| 1762/1762 [00:02<00:00, 869.21it/s]


In [61]:
print(np.mean(total_supply))

417.6592592592593


In [62]:
supply_demand_ratio=[]
for idx,demand in enumerate(demand_list):
    if demand['density']!=0:
        supply_demand_ratio.append(total_supply[idx]/demand['density'])
print(np.mean(supply_demand_ratio))
np.save("data/supply_demand_ratio_official.npy",supply_demand_ratio)

375.8320275716796


In [66]:
def FeasibilityCheck_one_slot(params):
    t,mega=params
    two_pi=2*np.pi
    total_supply=[]
    for idx,shell in enumerate(mega):
        T=T_list[idx]
        eta=coverage_eta(T)
        inc=inc_list[idx]
        for mid in range(shell["m"]):
            for nid in range(shell["n"]):
                r_set=[]
                a_sat=(two_pi/shell["m"]*mid-t/TE*two_pi)%two_pi
                b_sat=(two_pi/shell["n"]*nid+t/T*two_pi)%two_pi
                l_sat,p_sat=alpha_gamma_to_lambda_phi(a_sat,b_sat,inc)
                for idx,demand in enumerate(demand_list):
                    l_cell=demand['lat_lon'][1]
                    p_cell=demand['lat_lon'][0]
                    r=is_cover(l_sat,p_sat,l_cell,p_cell,eta) #用经纬度判断
                    if r:
                        S=S_set[idx]
                        r_set.append([idx,S,demand['density']])
                if len(r_set)!=0:
                    S_sum=np.sum([item[1] for item in r_set])
                    factor=user_per_sat/S_sum
                    r_set=[[item[0],item[1]*factor]for item in r_set]
                    total_supply.append(r_set)
    # check the demand
    supply=[0]*len(demand_list)
    for r_set in total_supply:
        for item in r_set:
            supply[item[0]]+=item[1]
    return supply

In [67]:
official_data=np.load("data/satisfied_mega_official_demand.npy",allow_pickle=True)
selected_data=official_data[-1][0]
T_list = np.array([satellite_period(shell["h"] * 1e3) for shell in selected_data])
inc_list = np.radians(np.array([shell["inc"] for shell in selected_data]))
total_supply=FeasibilityCheck_one_slot([0,selected_data])

In [68]:
supply_demand_ratio=[]
for idx,demand in enumerate(demand_list):
    if demand['density']!=0:
        supply_demand_ratio.append(total_supply[idx]/demand['density'])
print(np.mean(supply_demand_ratio))
np.save("data/supply_demand_ratio_official_megareduce.npy",supply_demand_ratio)

1319.6789211893495
